In [2]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/projects/cft_vlm/finetune')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from qwenvl.predict import gather_result
from pathlib import Path

output_dir = Path('/projects/cft_vlm/datasets/surgeryvid/results/test/Qwen2.5-VL-7B-Instruct/')
_ = gather_result(world_size=4, output_dir=output_dir)

2025-07-18 14:13:35,104 - qwenvl.predict - INFO - Inference complete. result saved to /projects/cft_vlm/datasets/surgeryvid/results/test/Qwen2.5-VL-7B-Instruct/results.jsonl
2025-07-18 14:13:35,105 - qwenvl.predict - INFO - Total processed samples: 2648


[{'id': '66bb0c8b-648a-4c7c-a307-9da630c97e7b',
  'video_id': '_TNRtl1_XUo',
  'video_url': 'https://www.youtube.com/watch?v=_TNRtl1_XUo',
  'video': '/projects/cft_vlm/datasets/surgeryvid/data/vid_processed/_TNRtl1_XUo_251_523.mp4',
  'timestamp': [251.0, 523.0],
  'question': 'What anatomical structure is identified for preservation during dissection?',
  'answer': 'Lingual nerve.',
  'video_metadata': {'original': {'duration': 272.0,
    'frame_count': 8160,
    'fps': 30.0,
    'resolution': {'width': 640, 'height': 480}}},
  'model_answer': 'The sciatic nerve is identified as the structure to be preserved during the dissection.'},
 {'id': '60aa9f25-3c93-4a8d-8831-aafd5475083b',
  'video_id': '_TNRtl1_XUo',
  'video_url': 'https://www.youtube.com/watch?v=_TNRtl1_XUo',
  'video': '/projects/cft_vlm/datasets/surgeryvid/data/vid_processed/_TNRtl1_XUo_251_523.mp4',
  'timestamp': [251.0, 523.0],
  'question': 'What is being ligated during the surgery?',
  'answer': 'Blood vessel.',
  '